# End of Day Portfolio Analysis

**Iain Muir, iam9ez@virginia.edu**

*Date: February 16th, 2021*

## Table of Contents

* **0. Import Libraries and Secrets** 
    * *Packages, Modules, Secrets, and Constants*
    * *Users Spreadsheet*
    * *Datapane Authorization*
    * *Configure Preferences*
* **1. Portfolio Summary**  
    * 1.1 Holdings Overview
    * 1.2 Historical Portfolio Performance
* **2. Portfolio News** 
* **3. Portfolio Analytics** 
* **4. Generate Reports**
* **5. Validate Reports** 
* **6. Export Reports** 

## 0. Import Libraries and Secrets

Python Packages

In [1]:
import pandas_datareader as pdr
from functools import reduce
import datapane as dp
import altair as alt
import pandas as pd
import numpy as np
import webbrowser
import datetime
import warnings
import json
import os

Modules

In [2]:
from robinhood import authenticate_, load_portfolio, robinhood_news, get_scroll_objects, ticker_toggle
from finnhub import quote, big_number, candles, candlestick, name_search, profile
from errors import ErrorHandler, Logging, get_error_info
from helpers import get_market_opens

Constants

In [3]:
from constants import ROOT

Users

In [4]:
users = pd.read_csv('users.csv')
users.shape

(1, 3)

Market Open

In [5]:
is_open = pd.read_csv(f'{ROOT}/Input/market_open.csv')
is_open['date'] = pd.to_datetime(is_open['date'], format='%Y-%m-%d').dt.date
is_open.shape

(805, 3)

Secrets

In [6]:
with open('secrets.json') as s:
    secrets = json.loads(s.read())

In [7]:
ROBINHOOD_USERS = secrets['robinhood']
FINNHUB_KEY = secrets['finnhub']
DATAPANE_KEY = secrets['datapane']

Datapane Authorization

In [8]:
version = !datapane --version

In [9]:
assert version[0].split()[2] == '0.13.2'

In [10]:
!datapane login --token=55010cebc170ecfbeddb82838c360776bf36f6be

Connected successfully to https://datapane.com as iainmuir


Preferences

In [11]:
warnings.simplefilter(action="ignore", category=pd.core.common.SettingWithCopyWarning)

## 1. Portfolio Summary

#### 1.1 Holdings Overview

In [12]:
def security_grouping(symbol, shares, id_, equity):
    """
    :param: symbol
    :param: shares
    :param: id_
    :param: equity
    
    :return:
    """
    
    # ----- Symbol and Shares -----
    info_content = f"""
        <p class='symbol'>
            {symbol}<br>
            <span class='shares_num'>{shares} </span><span class='shares_word'>shares</span>
        </p>
    """.strip()
    
    # -----  Intraday Chart  -----
    prices = candles(
        FINNHUB_KEY, 
        symbol if equity else f'BINANCE:{symbol}USDT', 
        resolution='1',
        type_='stock' if equity else 'crypto'
    )[-480:].reset_index(drop=True)
    
    begin, end = prices.iloc[0]['c'], prices.iloc[-1]['c']
    change = 'darkgreen' if end > begin else 'darkred'    
    delta = (end / begin - 1) * 100
    df = prices[['c', 't']]

    chart = alt.Chart(
        df
    ).mark_area(
        line={'color':change},
        color=alt.Gradient(
            gradient='linear',
            stops=[
                alt.GradientStop(color='white', offset=0),
                alt.GradientStop(color=change, offset=1)
            ]
        )
    ).encode(
        x=alt.X(
            "t:T",
            axis=alt.Axis(
                title="",
                labels=False
            )
        ),
        y=alt.Y(
            'c',
            axis=alt.Axis(
                title=""
            ),
            scale=alt.Scale(
                domain=[min(df['c']) - 1, max(df['c']) + 1]
            )
        )
    ).configure_view(
        strokeWidth=0,
        continuousHeight=75,
        continuousWidth=125
    ).configure_axis(
        grid=False, 
        domain=False
    )
    chart.save(f'{symbol}_chart.html')
    with open(f'{symbol}_chart.html') as c:
        chart_html = c.read()
    chart_content = chart_html[chart_html.find('<script>'): chart_html.rfind('</script>') + 9]
    chart_content = chart_content.replace('vegaEmbed("#vis', f'vegaEmbed("#vis{id_}')
    
    # -----  Intraday Change -----
    change_content = f"""
        <p class='change_{'up' if delta >= 0.0 else 'down'}'>
            {round(delta,  2)}%
        </p>
    """.strip()
    
    return f"""
        <div>
            {info_content}
        </div>
        <div id="vis{id_}" class="chart"></div>
        {chart_content}
        <div>
            {change_content}
        </div>
    """.strip()

In [13]:
def security_html(symbols, quantities, id_, equity=True):
    """
    
    :param: symbols
    :param: quantities
    :param: id_
    :param: equity
    :return:
    """
    
    groups = list(map(
        lambda s, q, i: security_grouping(s, q, i, equity),
        symbols, quantities, id_
    ))
    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <style type='text/css'>
            .grid-container {
                display: grid;
                grid-template-columns: auto auto auto;
                border-top: 0px solid black;
                border-left: 0px solid black;
            }
            .grid-container > div {
                border-bottom: 0px;
                border-right: 0px;
            }
            .chart {
                display: block;
                margin: 0 auto;
            }
            .symbol {
                font-size: 20px;
                font-weight: bold;
                color: #000000;
                padding: 15px 0;
                float: right;
                text-align: center;
            }
            .shares_num {
                font-size: 12px;
                color: #36454F;
            }
            .shares_word {
                font-size: 10px;
                color: #36454F;
            }
            .error {
                color: red;
            }
            .change_up {
                font-size: 17px;
                font-weight: bold;
                color: darkgreen;
                padding: 20px 0;
                float: left;
            }
            .change_down {
                font-size: 17px;
                font-weight: bold;
                color: darkred;
                padding: 20px 0;
                float: left;
            }
        </style>
        <script type="text/javascript" src="https://cdn.jsdelivr.net/npm//vega@5"></script>
        <script type="text/javascript" src="https://cdn.jsdelivr.net/npm//vega-lite@4.17.0"></script>
        <script type="text/javascript" src="https://cdn.jsdelivr.net/npm//vega-embed@6"></script>
    </head>
    <body>
        <div class='grid-container'>
        """ + '\n'.join(groups) + """
        </div>
    </body>
    </html>
    """.strip()

    return dp.HTML(html)

#### 1.2 Portfolio Historical

Get Transactions

In [14]:
def get_referrals(client):
    """
    
    :param: client
    :return:
    """
    
    referrals = client.account.get_referrals()
    
    # Stock Referrals
    referral_stock = pd.concat(
        [pd.DataFrame(referral['reward']['stocks']) for referral in referrals]
    )
    referral_stock = pd.DataFrame(
        [
            [s[2], s[10], '', 'buy', 0.0, s[3], s[4]]
            for s in referral_stock.values
        ]
    )
    
    # Cash Referrals
    referral_cash = pd.concat(
        [pd.DataFrame(referral['reward']['cash']) for referral in referrals]
    )
    referral_cash = pd.DataFrame(
        [
            ['Cash', s[10], '', 'deposit', 0.0, 1.0, s[4]]
            for s in referral_cash.values
        ]
    )
    
    return referral_stock, referral_cash

In [15]:
def get_portfolio_transactions(client):
    """
    
    :param: client
    :return:
    """
    
    # EXPORT ALL TRADES
    client.export.export_completed_stock_orders(
        dir_path=ROOT,
        file_name='stock_orders.csv'
    )
    client.export.export_completed_crypto_orders(
        dir_path=ROOT,
        file_name='crypto_orders.csv'
    )
    client.export.export_completed_option_orders(
        dir_path=ROOT,
        file_name='option_orders.csv'
    )
    
    # LOAD ALL TRADES
    stock_orders = pd.read_csv('stock_orders.csv')
    crypto_orders = pd.read_csv('crypto_orders.csv')
    option_orders = pd.read_csv('option_orders.csv')
    
    trades = pd.concat(
        [stock_orders, crypto_orders]
    )
    
    # DELETE ALL TRADES
    os.remove('stock_orders.csv')
    os.remove('crypto_orders.csv')
    os.remove('option_orders.csv')

    return stock_orders, crypto_orders, option_orders

Reverse-Engineer Portfolio

In [16]:
def theoretical_candles(symbol, start, present):
    """
    :param: symbol
    :param: start
    :param: present
    :return:
    """
    

In [17]:
def reverse_engineer(client, trades, equity_symbols, crypto_symbols, referrals):
    """
    
    :param: client
    :param: trades
    :param: equity_symbols
    :param: crypto_symbols
    :param: referrals
    :return:
    """
    # LOAD AND CLEAN ALL TRANSFERS
    transfers = pd.DataFrame(
        client.account.get_bank_transfers()
    )
    transfers = transfers.apply(
        lambda x: pd.Series(
            ['Cash', x[15], None, x[7], x[9], 1, x[6]]
        ),
        axis=1
    )
    transfers.columns = trades.columns
    
    # CONCATENATE AND CLEAN ALL TRANSACTIONS
    transactions = pd.concat(
        [trades, transfers]
    )
    transactions['date'] = pd.to_datetime(
        pd.Series(transactions['date'].str[:10]),
        format='%Y-%m-%d'
    ).dt.date
    transactions = transactions.sort_values(
        by='date', 
        ascending=True
    ).reset_index(drop=True)
    transactions['fees'] = transactions['fees'].astype(float)
    transactions['quantity'] = transactions['quantity'].astype(float)
    transactions['average_price'] = transactions['average_price'].astype(float)
    
    # REVERSE ENGINEER PORTFOLIO WEIGHTS
    symbols = transactions.symbol.unique().tolist()
    
    # Initial and Present Dates
    start = transactions.iloc[0]['date']
    present = datetime.date.today()
    days = (present - start).days
    
    portfolio_weights = []
    w = {
        symbol: 0.0 for symbol in symbols
    }
    w['Cash'] = 0.0
    
    # Locate all Transactions from the start to present
    # ---> if transaction on that day, update security weight and resulting remaining cash
    # ---> accounts for bank transfers, withdrawals, and Robinhood referrals
    for date in pd.date_range(start, present):
        result = transactions.loc[transactions['date'] == date.date()]
        if len(result) != 0:
            for r in result.values:
                symbol, d, type_, direction, fees, quantity, price = r
                if symbol == 'Cash':
                    w['Cash'] += quantity * price * (-1 if direction == 'withdraw' else 1)
                else:
                    w[symbol] += quantity * (-1 if direction == 'sell' else 1)
                    if type_ != '':
                        w['Cash'] += quantity * price * (-1 if direction == 'buy' else 1)
                    
        portfolio_weights.append(list(w.values()))
    historical_weights = pd.DataFrame(
        portfolio_weights,
        columns=symbols,
        index=pd.date_range(start, present)
    )
    
    # Add to Market Open Dataset if out of range
    if start < is_open.iloc[0]['date']:
        get_market_opens(robinhood, is_open.iloc[0]['date'], start)
    elif present > is_open.iloc[-1]['date']:
        get_market_opens(robinhood, present, is_open.iloc[-1]['date'])
    
    # Subset for Open Market Dats
    market_open = is_open.loc[
        (is_open['date'] >= start) & (is_open['date'] <= present)
    ]
    market_open_dates = market_open['date']
    market_open = market_open['is_open']
    historical_weights = historical_weights[market_open.values]
    
    cash = historical_weights['Cash']
    historical_weights = historical_weights.drop(
        'Cash', axis=1
    )
    historical_weights.index.name = 'date'
    
    # Duplicate first symbol for timestamp column
    equity_symbols_, crypto_symbols_ = equity_symbols, crypto_symbols
    equity_symbols_.insert(0, equity_symbols[0])
    crypto_symbols_.insert(0, crypto_symbols[0])
    
    # Query Historical Prices
    equity_historical_prices = pd.concat(
        [
            pd.Series(
                candles(
                    FINNHUB_KEY, 
                    symbol, 
                    years=(start, present),
                    type_='stock'
                )['t' if i == 0 else 'c']
            )
            for i, symbol in enumerate(equity_symbols_)
        ],
        axis=1
    )
    equity_historical_prices.columns = ['date'] + equity_symbols[1:]

    crypto_historical_prices = pd.concat(
        [
            pd.Series(
                candles(
                    FINNHUB_KEY, 
                    f'BINANCE:{symbol}T', 
                    years=(start, present),
                    type_='crypto'
                )['t' if i == 0 else 'c']
            )
            for i, symbol in enumerate(crypto_symbols_)
        ],
        axis=1
    )
    crypto_historical_prices.columns = ['date'] + crypto_symbols[1:]

    historical_prices = pd.merge(
        equity_historical_prices, 
        crypto_historical_prices,
        how='left',
        on='date'
    )
    historical_prices['date'] = pd.to_datetime(
        historical_prices['date'], 
        unit='s'
    ).dt.date
    historical_prices = historical_prices.set_index('date')

    return historical_prices, historical_weights, cash

## 2. Portfolio News

In [18]:
def get_portfolio_news(client, tickers):
    """
    
    :param: client
    :param: tickers
    :return: 
    """
    portfolio_news = list(map(
        lambda t: robinhood_news(client, t), tickers
    ))
    portfolio_news = [
        article for ticker_news in portfolio_news for article in ticker_news
    ]
    
    return portfolio_news

## 3. Portfolio Analytics

In [19]:
sp500 = pdr.get_data_yahoo('^GSPC')['Close']
sp500 = np.log(
    sp500 / sp500.shift()
).values

In [20]:
def calculate_beta(sp500, ticker, years=5, equity=True):
    """
    
    :param: sp500
    :param: ticker
    :param: years
    :param: equity
    :return:
    """
    c = candles(
        FINNHUB_KEY, 
        ticker if equity else f'BINANCE:{ticker}USDT', 
        years=years,
        type_='stock' if equity else 'crypto'
    )['c']

    if sp500.shape[0] > c.shape[0]:
        sp500 = sp500[:c.shape[0]]
    else:
        c = c[:sp500.shape[0]]
    
    log_r = np.log(
        c / c.shift()
    ).values
    df = pd.DataFrame({
        "sp500": sp500,
        ticker: log_r
    })
    cov_ = df.cov()
    var_ = df.sp500.var()
    beta = (cov_/var_).loc['sp500', ticker]
    
    return beta

## 4. Generate Reports

In [21]:
TODAY = datetime.date.today()

In [22]:
def build_header(user):
    """
    
    :param: user
    :return:
    """
    
    return dp.HTML(
        """
        <html>
            <style type='text/css'>
                @keyframes rotate {
                    0%   {color: #0BDA51;}
                    15%  {color: #32CD32;}
                    30%  {color: #4CBB17;}
                    60%  {color: #008000;}
                    75%  {color: #4CBB17;}
                    90%  {color: #32CD32;}
                    100% {color: #0BDA51;}
                }
                h1 {
                    color: #0BDA51;
                    animation-name: rotate;
                    animation-duration: 4s;
                    animation-iteration-count: infinite;
                }
            </style>
            <h1>Portfolio Analysis</h1>
            <b>User:</b> """ + user + """<br><br>
            <b>Generated:</b> """ + TODAY.strftime('%A, %B %d, %Y') + """
        </html>
        """.strip()
    )

In [23]:
def upload_report(r, user):
    """
    
    :param: r
    :param: user
    :return:
    """
    try:
        r.upload(
            name='Portfolio Analysis', 
            open=False
        )
    except requests.exceptions.HTTPError:
        print(
            ErrorHandler("Report Upload Error; FATAL.", *get_error_info())
        )
        time.sleep(5)

        r.upload(
            name='Portfolio Analysis', 
            open=False
        )
        
    r.save(
        path=f'{ROOT}/Portfolio-Analysis.html'
    )
    webbrowser.open(
        r.web_url
    )

In [24]:
def generate_report(user):
    """
    
    :param: user
    :return:
    """

    user_info = users.loc[users.short_name == user]
    name, _, email = user_info.values[0]
    
    print(f"---------- Generating {name.title()}'s Report ----------\n")
    
    # ---------- ROBINHOOD AUTHENTICATION ----------
    username, password = ROBINHOOD_USERS[user]
    robinhood = authenticate_(username, password)
    print("   1. Successful Robinhood Authentication")
    
    # GET PORTFOLIO TRANSACTIONS
    # ---> all transactions on account
    stocks, crypto, options = get_portfolio_transactions(robinhood)
    all_stock_symbols = stocks.symbol.unique().tolist()
    all_crypto_symbols = crypto.symbol.unique().tolist()
    all_option_symbols = options.chain_symbol.unique().tolist()
    
    # BUILD HOLDINGS
    # ---> only open positions
    tickers, data, profile = load_portfolio(robinhood)
    stock_symbols, etf_symbols, crypto_symbols = tickers
    stock_open, etf_open, crypto_open = data
    flat_tickers = np.array(
        tickers, dtype=object
    ).flatten().tolist()
    stock_quantities = stock_open.quantity.astype(float).tolist()
    etf_quantities = etf_open.quantity.astype(float).tolist()

    # ---> Intermediate Crypto Quote Lookup
    crypto_quotes = list(map(
        lambda symbol: float(robinhood.crypto.get_crypto_quote(symbol)['mark_price']),
        crypto_symbols
    ))
    crypto_quantities = crypto_open.quantity_available.astype(float).tolist()
    crypto_value = (np.multiply(crypto_quantities, crypto_quotes)).sum()
    
    start = profile['start_date']
    mkt_value, prev_mkt_value = profile['market_value'], profile['last_core_market_value']
    mkt_value, prev_mkt_value = float(mkt_value), float(prev_mkt_value)
    delta_pct = mkt_value / prev_mkt_value - 1
    cash = float(profile['withdrawable_amount'])
    portfolio_value = mkt_value + crypto_value + cash
    print("   2. Built Holdings and Retrieved Historical Transactions")
    
#     # ---------- PORTFOLIO SUMMARY ----------
#     # ---> Individual Securities
#     summary_blocks = []
#     if len(stock_symbols) > 0:
#         stock_html = security_html(
#             stock_symbols, stock_quantities, range(1, len(stock_symbols) + 1), equity=True
#         )
#         summary_blocks.extend(
#             [dp.Text(f"### Equities"), stock_html]
#         )
#     if len(etf_symbols) > 0:
#         etf_html = security_html(
#             etf_symbols, etf_quantities, range(1, len(etf_symbols) + 1), equity=True
#         )
#         summary_blocks.extend(
#             [dp.Text(f"### Exchange Traded Funds"), etf_html]
#         )
#     if len(crypto_symbols) > 0:
#         crypto_html = security_html(
#             crypto_symbols, crypto_quantities, range(1, len(crypto_symbols) + 1), equity=False
#         )
#         summary_blocks.extend(
#             [dp.Text(f"### Cryptocurrencies"), crypto_html]
#         )
        
    # ---> Overall Portfolio
    trades = pd.concat(
        [stocks, crypto]
    )
    
    # Account for Referrals
    referral_stock, referral_cash = get_referrals(robinhood)
    referrals = pd.concat(
        [referral_stock, referral_cash]
    )
    referrals.columns = trades.columns
    
    trades = pd.concat(
        [trades, referrals]
    )
    
    # Reverse Engineer Portfolio Historical Value
    prices, weights, cash = reverse_engineer(
        robinhood, 
        trades, 
        all_stock_symbols, 
        all_crypto_symbols, 
        referrals
    )
    return prices, weights, cash
    values = prices * weights
    values['Cash'] = cash
    portfolio_value = values.sum(
        axis=0
    )
    return 
    
    summary = dp.Group(
        blocks=[
            dp.Group(blocks=[*summary_blocks]),
            dp.Text('Portfolio Summary')
        ],
        columns=2,
        label='Overview'
    )
    print("   3. Completed Portfolio Summary")
    
    # ----------   PORTFOLIO NEWS  ----------
    
    news = get_portfolio_news(
        robinhood,
        stock_symbols
    )
    news = dp.Group(
        blocks=news,
        columns=2,
        label='News'
    )
    print("   4. Aggregated Portfolio News")
    
    # ---------- PORTFOLIO ANALYSIS ---------
    analysis = dp.Text(
        "None",
        label='Analysis'
    )
    
    # ----------   MISCELLAENOUS   ----------
    header = build_header(name)
    credits = dp.Text(
        "Report built by Iain Muir."
    )
    
    # BUILD REPORT
    report = dp.Report(
        blocks=[
            header,
            dp.Divider(),
            dp.Select(
                blocks=[
                    summary, news, analysis
                ],
                type=dp.SelectType.TABS,
                label='main_select'
            ),
            dp.Divider(),
            credits
        ]
    )
    print("   5. Succesfully Build Datapane Report\n\n")
    
    # REPORT UPLOAD
    upload_report(report, user)
    
    # Remove Altair Chart HTML
    for file in os.listdir(os.getcwd()):
        if "." not in file:
            continue
        if '.html' in file and '_chart' in file:
            os.remove(file)

    return "COMPLETE"

In [25]:
%%time

status = users.short_name.apply(
    lambda user: generate_report(user)
)

---------- Generating Iain Muir's Report ----------

ERROR: There was an issue loading pickle file. Authentication may be expired - logging in normally.


Exception: Unable to log in with provided credentials.

In [26]:
p = status[0][0]
p = p.set_index('date')
p

NameError: name 'status' is not defined

In [ ]:
p.isnull().sum()

In [ ]:
q = status[0][1]
q.index.name = 'date'
q

In [ ]:
values = p * q
values

In [ ]:
values['Cash'] = status[0][2]
values

## 5. Validate Reports

In [ ]:
def validate_report(user, status):
    """
    
    :param: user
    :param: status
    :return:
    """
    if status == 'COMPLETE':
        print(f"{user}'s Report: SUCCESS!")
        Logging.write_success_to_log(user)
    else:
        print(
            ErrorHandler(
                f"Unable to Generate {user}'s Report", *get_error_info()
            )
        )

In [ ]:
validate = list(map(
    lambda user, status: validate_report(user, status),
    users.name,
    status
))

## 6. Export Reports

In [ ]:
# TODO GMAIL API?